In [114]:
import pandas as pd
import numpy as np
import datetime
import os 

In [115]:
#read in the latest dataset
time_series = pd.read_csv("../../../databases/JHU/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,145,145,145,145,149,149,151,151,151,151
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,15,15,15,16,16,19
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1757,1808,1843,1924,1968,2031,2156,2173,2198,2256
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,66,66,66,66,66,68,68,69,69


In [116]:
#keep only data from NC counties
time_series = time_series[time_series["Province_State"] == "North Carolina"]
time_series = time_series[time_series["Admin2"] != "Out of NC"]
time_series = time_series[time_series["Admin2"] != "Unassigned"]
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
1895,84037001,US,USA,840,37001.0,Alamance,North Carolina,US,36.043470,-79.399761,...,121,124,128,128,128,143,149,152,173,183
1896,84037003,US,USA,840,37003.0,Alexander,North Carolina,US,35.922380,-81.177519,...,4,4,4,5,6,8,8,10,12,12
1897,84037005,US,USA,840,37005.0,Alleghany,North Carolina,US,36.493609,-81.128570,...,6,6,6,6,7,7,7,7,8,8
1898,84037007,US,USA,840,37007.0,Anson,North Carolina,US,34.974032,-80.099533,...,29,30,30,30,31,33,37,38,38,38
1899,84037009,US,USA,840,37009.0,Ashe,North Carolina,US,36.432962,-81.498627,...,5,5,5,5,5,6,7,8,13,13


In [117]:
#define a dataframe with just the county information
nc_counties = time_series[["FIPS", "Admin2","Province_State","Lat","Long_"]]
nc_counties.head()

,FIPS,Admin2,Province_State,Lat,Long_
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570
1898,37007.0,Anson,North Carolina,34.974032,-80.099533
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627


In [118]:
#define variable for latest day of data to be pulled
startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days
offset

122

In [119]:
#create a dataframe that is just the number of total cases by county for each day
dates = time_series.iloc[:,53:offset]
dates.head()

,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
1895,0,0,0,0,0,0,0,0,0,0,...,121,124,128,128,128,143,149,152,173,183
1896,0,0,0,0,0,0,0,0,0,0,...,4,4,4,5,6,8,8,10,12,12
1897,0,0,0,0,0,0,0,0,0,0,...,6,6,6,6,7,7,7,7,8,8
1898,0,0,0,0,0,0,0,0,0,0,...,29,30,30,30,31,33,37,38,38,38
1899,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,6,7,8,13,13


In [120]:
#create an empty dataframe that will be filled with data needed to calculate new cases and 14-day rolling average
nc_time_series = pd.DataFrame(columns = ["FIPS","Admin2","Province_State","Lat","Long_", "Confirmed"
                                             "Yesterday", "Day_3","Day_4","Day_5","Day_6","Day_7",
                                             "Day_8","Day_9","Day_10","Day_11","Day_12","Day_13","Day_14","Date"])

In [121]:
#pull the values to fill the cells for calculating new cases and 14-day rolling average
cols = dates.columns

for i in range(14,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Yesterday", "Day_3","Day_4","Day_5","Day_6","Day_7",
                                             "Day_8","Day_9","Day_10","Day_11","Day_12","Day_13","Day_14","Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Yesterday"] = dates.iloc[:,i-1]
    one_day["Day_3"] = dates.iloc[:,i-2]
    one_day["Day_4"] = dates.iloc[:,i-3]
    one_day["Day_5"] = dates.iloc[:,i-4]
    one_day["Day_6"] = dates.iloc[:,i-5]
    one_day["Day_7"] = dates.iloc[:,i-6]
    one_day["Day_8"] = dates.iloc[:,i-7]
    one_day["Day_9"] = dates.iloc[:,i-8]
    one_day["Day_10"] = dates.iloc[:,i-9]
    one_day["Day_11"] = dates.iloc[:,i-10]
    one_day["Day_12"] = dates.iloc[:,i-11]
    one_day["Day_13"] = dates.iloc[:,i-12]
    one_day["Day_14"] = dates.iloc[:,i-13]
    one_day["Date"] = cols[i]
    nc_time = nc_counties.join(one_day)
    nc_time_series = nc_time_series.append(nc_time)
#    county_time.append(one_day)
#    print(one_day)
nc_time_series.head()


,FIPS,Admin2,Province_State,Lat,Long_,ConfirmedYesterday,Day_3,Day_4,Day_5,Day_6,...,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Date,Confirmed,Yesterday
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761,NaN,0,0,0,0,...,0,0,0,0,0,0,0,3/18/20,0.0,0.0
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519,NaN,0,0,0,0,...,0,0,0,0,0,0,0,3/18/20,0.0,0.0
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570,NaN,0,0,0,0,...,0,0,0,0,0,0,0,3/18/20,0.0,0.0
1898,37007.0,Anson,North Carolina,34.974032,-80.099533,NaN,0,0,0,0,...,0,0,0,0,0,0,0,3/18/20,0.0,0.0
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627,NaN,0,0,0,0,...,0,0,0,0,0,0,0,3/18/20,0.0,0.0


In [122]:
nc_time_series.reset_index(inplace = True)
nc_time_series.drop("index", axis = 1, inplace = True)

In [123]:
nc_time_series["Date"] = pd.to_datetime(nc_time_series.Date)
nc_time_series.dtypes

FIPS                         float64
Admin2                        object
Province_State                object
Lat                          float64
Long_                        float64
ConfirmedYesterday            object
Day_3                         object
Day_4                         object
Day_5                         object
Day_6                         object
Day_7                         object
Day_8                         object
Day_9                         object
Day_10                        object
Day_11                        object
Day_12                        object
Day_13                        object
Day_14                        object
Date                  datetime64[ns]
Confirmed                    float64
Yesterday                    float64
dtype: object

In [124]:
nc_time_series["Date"] = nc_time_series["Date"].dt.to_period("D")

In [125]:
nc_time_series.tail()

,FIPS,Admin2,Province_State,Lat,Long_,ConfirmedYesterday,Day_3,Day_4,Day_5,Day_6,...,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Date,Confirmed,Yesterday
5495,37191.0,Wayne,North Carolina,35.364381,-78.000273,NaN,726,715,699,688,...,685,677,674,658,655,651,641,2020-05-11,752.0,743.0
5496,37193.0,Wilkes,North Carolina,36.208413,-81.162329,NaN,202,198,186,147,...,115,92,59,42,33,26,22,2020-05-11,242.0,239.0
5497,37195.0,Wilson,North Carolina,35.706255,-77.917978,NaN,203,199,192,192,...,180,175,173,165,165,159,155,2020-05-11,216.0,212.0
5498,37197.0,Yadkin,North Carolina,36.162226,-80.662394,NaN,49,39,33,27,...,24,23,23,17,17,17,15,2020-05-11,54.0,54.0
5499,37199.0,Yancey,North Carolina,35.900792,-82.312372,NaN,8,7,7,1,...,1,1,0,0,0,0,0,2020-05-11,9.0,9.0


In [126]:
#create and calculate a column for the number of new cases each day
nc_time_series["New_Cases"] = nc_time_series.apply(lambda row: row.Confirmed - row.Yesterday, axis=1)

In [127]:
#create and calculate a column with a rolling average
nc_time_series["Rolling_Avg"] = nc_time_series.apply(
    lambda row: (row.Confirmed + row.Yesterday + row.Day_3+row.Day_4
                 +row.Day_5+row.Day_6+row.Day_7+row.Day_8+row.Day_9+
                 row.Day_10+row.Day_11+row.Day_12+row.Day_13+row.Day_14)/14, axis = 1)

In [128]:
#now that Rolling_Avg is calculated, get rid of the unneeded calculation columns
nc_time_series = nc_time_series[["FIPS", "Admin2", "Province_State", "Confirmed", 
                                         "New_Cases", "Rolling_Avg", "Date"]]
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
5495,37191.0,Wayne,North Carolina,752.0,9.0,689.428571,2020-05-11
5496,37193.0,Wilkes,North Carolina,242.0,3.0,123.642857,2020-05-11
5497,37195.0,Wilson,North Carolina,216.0,4.0,183.428571,2020-05-11
5498,37197.0,Yadkin,North Carolina,54.0,0.0,29.785714,2020-05-11
5499,37199.0,Yancey,North Carolina,9.0,0.0,3.142857,2020-05-11


In [129]:
nc_time_series.sort_values(["Admin2", "Date"], inplace = True)
nc_time_series.reset_index(inplace = True)
nc_time_series.drop("index", axis = 1, inplace = True)
nc_time_series.head(5)

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
0,37001.0,Alamance,North Carolina,0.0,0.0,0.000000,2020-03-18
1,37001.0,Alamance,North Carolina,0.0,0.0,0.000000,2020-03-19
2,37001.0,Alamance,North Carolina,1.0,1.0,0.071429,2020-03-20
3,37001.0,Alamance,North Carolina,2.0,1.0,0.214286,2020-03-21
4,37001.0,Alamance,North Carolina,2.0,0.0,0.357143,2020-03-22


In [130]:
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
5495,37199.0,Yancey,North Carolina,7.0,6.0,0.785714,2020-05-07
5496,37199.0,Yancey,North Carolina,7.0,0.0,1.285714,2020-05-08
5497,37199.0,Yancey,North Carolina,8.0,1.0,1.857143,2020-05-09
5498,37199.0,Yancey,North Carolina,9.0,1.0,2.500000,2020-05-10
5499,37199.0,Yancey,North Carolina,9.0,0.0,3.142857,2020-05-11


In [131]:
#add a column to calculate rolling average of new cases
nc_time_series["Temp"] = nc_time_series.Rolling_Avg.shift(1)
nc_time_series.head()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date,Temp
0,37001.0,Alamance,North Carolina,0.0,0.0,0.000000,2020-03-18,NaN
1,37001.0,Alamance,North Carolina,0.0,0.0,0.000000,2020-03-19,0.000000
2,37001.0,Alamance,North Carolina,1.0,1.0,0.071429,2020-03-20,0.000000
3,37001.0,Alamance,North Carolina,2.0,1.0,0.214286,2020-03-21,0.071429
4,37001.0,Alamance,North Carolina,2.0,0.0,0.357143,2020-03-22,0.214286


In [132]:
#create and calculate row with rolling average of new cases
nc_time_series["New_Case_Roll_Avg"] = nc_time_series.apply(lambda row: row.Rolling_Avg - row.Temp, axis=1)
nc_time_series = nc_time_series[nc_time_series["Date"] != "2020-03-18"]
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date,Temp,New_Case_Roll_Avg
5495,37199.0,Yancey,North Carolina,7.0,6.0,0.785714,2020-05-07,0.285714,0.500000
5496,37199.0,Yancey,North Carolina,7.0,0.0,1.285714,2020-05-08,0.785714,0.500000
5497,37199.0,Yancey,North Carolina,8.0,1.0,1.857143,2020-05-09,1.285714,0.571429
5498,37199.0,Yancey,North Carolina,9.0,1.0,2.500000,2020-05-10,1.857143,0.642857
5499,37199.0,Yancey,North Carolina,9.0,0.0,3.142857,2020-05-11,2.500000,0.642857


In [133]:
#remove Temp column
nc_time_series = nc_time_series[["FIPS","Admin2","Province_State","Confirmed","Rolling_Avg",
                                       "New_Cases","New_Case_Roll_Avg","Date"]]
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,Rolling_Avg,New_Cases,New_Case_Roll_Avg,Date
5495,37199.0,Yancey,North Carolina,7.0,0.785714,6.0,0.500000,2020-05-07
5496,37199.0,Yancey,North Carolina,7.0,1.285714,0.0,0.500000,2020-05-08
5497,37199.0,Yancey,North Carolina,8.0,1.857143,1.0,0.571429,2020-05-09
5498,37199.0,Yancey,North Carolina,9.0,2.500000,1.0,0.642857,2020-05-10
5499,37199.0,Yancey,North Carolina,9.0,3.142857,0.0,0.642857,2020-05-11


In [134]:
#add a column with a unique value that can be used to establish a relationship with other datasheets
nc_time_series["Unique"] = nc_time_series["Admin2"]+", "+nc_time_series["Date"].astype(str)
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,Rolling_Avg,New_Cases,New_Case_Roll_Avg,Date,Unique
5495,37199.0,Yancey,North Carolina,7.0,0.785714,6.0,0.500000,2020-05-07,"Yancey, 2020-05-07"
5496,37199.0,Yancey,North Carolina,7.0,1.285714,0.0,0.500000,2020-05-08,"Yancey, 2020-05-08"
5497,37199.0,Yancey,North Carolina,8.0,1.857143,1.0,0.571429,2020-05-09,"Yancey, 2020-05-09"
5498,37199.0,Yancey,North Carolina,9.0,2.500000,1.0,0.642857,2020-05-10,"Yancey, 2020-05-10"
5499,37199.0,Yancey,North Carolina,9.0,3.142857,0.0,0.642857,2020-05-11,"Yancey, 2020-05-11"


In [135]:
nc_time_series.to_excel("../clean_data/nc_rolling_avg.xlsx")